In [17]:
import cv2
import numpy as np
from numpy import shape
import matplotlib.pyplot as plt 
from image_extractor import select_feature, feature_extract

# image_dir  = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Agglutinated/image (2).JPG'
# image_dir  = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Brittle/image (87).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Plated_Purple/image (10).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Compartmentalized_Purple/image (100).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Compartmentalized_White/image (24).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Plated_White/image (72).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Plated_Slaty/image (71).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Compartmentalized_Slaty/image (56).JPG'
image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Moldered/image (7).JPG'

plt.rcParams['figure.dpi'] = 102.46 

In [18]:
# features = select_feature(image_dir)
# print(type(features))

In [19]:
extractor = feature_extract()
image_bgr = cv2.imread(image_dir)
extractor.process(image_bgr)


In [20]:
print((extractor.structure))

[ 8.45993072e+01  1.93394755e+02  3.23050771e+01  3.28259911e+01
  8.27227265e+01  1.57991824e+02  4.72842601e+00  1.86787442e+01
  1.11187347e+02  2.53394417e+02  3.25723362e+00  2.06215725e+01
  1.09809029e+02  2.09552597e+02  1.45178871e+01  2.02295160e+01
  1.06831529e+02  1.98938780e+02  2.23971130e+01  4.16232838e+00
  8.67503717e+01  1.97904362e+02  2.40834695e+01  4.83319619e+00
  3.24667856e+01  7.35067192e+01  3.74572986e+01  2.56341809e+01
  3.05799531e+01  7.74853153e+01  1.32349761e+00  6.73165500e+00
  9.37264252e+00  4.61856003e+01  3.42766762e-01  3.97843170e+00
  1.14938871e+01  5.80138207e+01  1.88258764e+01  7.87830643e+00
  3.16832782e+01  8.74720214e+01  9.47738238e+00  1.16521683e+00
  3.21654426e+01  8.91681794e+01  1.65249451e+01  1.61792879e+00
  1.06621323e-02  2.69748541e-01  1.18284905e+00  9.95685807e-01
  9.50608837e-02 -6.94258413e-02  4.80441775e-01 -5.24137026e-01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3.34649424e-01 -8.02741

In [21]:
print(extractor.n2)

17


In [22]:
print(extractor.mold)

[ 1.15294118e+01  1.14117647e+01  1.28089325e+02  1.18188471e+02
  7.24575092e+01  3.37740711e+01  3.67732873e+01  2.96727637e+01
 -2.88585054e-01 -2.00195990e-01 -1.20269816e-01 -5.07565999e-01
 -6.96226146e-01 -5.44137495e-01  1.02160062e+01  8.97748048e+00
  1.20916863e+01  1.26747949e+01  7.54295325e+00  1.33799553e+01
  1.27857133e+01  5.93597612e+00  5.88739367e-01  5.36435391e-01
  5.29313723e-01  7.75766758e-01  4.79354288e-01  7.93316900e-01
  2.15973966e+00  1.88283984e+00 -7.15697552e-01 -4.04153524e-01
  1.60164702e-01  1.04147581e+00  4.80009693e-01  4.72527970e-01
 -1.13202362e-01 -3.07105323e-01 -8.18112013e-01  1.56958517e+00
  7.12390541e-01  2.85736907e+00  4.11365036e+00  2.37753922e+00
  4.93834494e-01 -1.11509798e+00 -7.18058569e-01 -1.74685309e-01
 -1.15930378e+00 -1.09169868e+00 -5.68677379e-01 -4.31230772e-01
 -7.83140419e-02  1.90056121e+00  3.33414538e-01  8.01218075e+00]


In [23]:
print(extractor.overall_geometry)

[167, 84, 169, 346, 44476.5, 1010.3056902885437, 161.39376831054688, 352.6984558105469, 1.8186364979124459, 46632.5, 173.04312133789062, 0.9537661502171233, 0.760620104661901, 7.729956530544822]


In [24]:
print(extractor.overall_statistic)

[72.14122    60.432976   30.720839   49.568996   43.610176   24.38077
 -0.14772646  0.15740381  0.86236995 -0.9093483  -0.43672904  1.8085557 ]
